In [4]:
!pip install kaggle

upload your kaggle json file


In [11]:
#!mkdir ~p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing Twitter Sentiment dataset

In [13]:
# API to fetch the dataset from kaggle
#!/bin/bash
!kaggle datasets download kazanova/sentiment140



Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 999MB/s]


In [14]:
# Extract zip file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing Dependencies

In [17]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
# print the stopwords in english
print(stopwords.words('english'))


['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data processing

In [20]:
# Loading the dataset from csv to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')


In [21]:
# checking the number of rows and columns
print(twitter_data.shape)

(1599999, 6)


In [22]:
# print first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [23]:
# naming the columns and reading the dataset again
column_names = ['target', 'id', 'data', 'flag', 'user', 'text']

In [24]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names,encoding = 'ISO-8859-1')


In [25]:
# checking the number of rows and columns
print(twitter_data.shape)

(1600000, 6)


In [26]:
# print first 5 rows of the dataframe
twitter_data.head()

,target,id,data,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [27]:
# checking missing values
twitter_data.isnull().sum()

,0
target,0
id,0
data,0
flag,0
user,0
text,0


In [29]:
# checking the distribution of target column
twitter_data['target'].value_counts()        #if it is not equal , we need to do sampling

,count
target,
0,800000
4,800000


convert the target '4' to '1'

In [36]:
twitter_data.replace({'target':{4:1}}, inplace = True)

In [37]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0  ------> Negative Tweet

1  ------> Positive Tweet

**Stemming**

Stemming is the process of reducing a word into key word/ root word

Ex. Actor , actress, acting = act

In [38]:
port_stem = PorterStemmer()

In [47]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [51]:
twitter_data.head()

,target,id,data,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [52]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [53]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [54]:
#separating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [55]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [56]:
print(Y)

[0 0 0 ... 1 1 1]


Train Test Split

In [80]:
X_train, X_test, Y_train, Y_test = train_test_split(X , Y, test_size=0.2, stratify=Y,random_state=2)

In [59]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [60]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [61]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


**Feature Extraction**

In [81]:
# Converting  the textual data to numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [82]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453092 stored elements and shape (1280000, 461488)>
  Coords	Values
  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (12

In [66]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2363913 stored elements and shape (320000, 179187)>
  Coords	Values
  (0, 104606)	0.4459147041194029
  (0, 51603)	0.25774294336856257
  (0, 107797)	0.17983918796121573
  (0, 158685)	0.3174206399662331
  (0, 26316)	0.2701195948137176
  (0, 163496)	0.180932348330391
  (0, 67046)	0.28383380524479984
  (0, 11837)	0.16338171192787918
  (0, 150603)	0.2214371376881977
  (0, 5650)	0.17295837060294847
  (0, 154815)	0.3501587145032445
  (0, 41330)	0.3621075009813407
  (0, 53904)	0.23662705066222536
  (1, 5650)	0.181974929607951
  (1, 2443)	0.2650067441796073
  (1, 99057)	0.2461083000957446
  (1, 141842)	0.21193862113578937
  (1, 134752)	0.4128221547743352
  (1, 83954)	0.35024490258566765
  (1, 56881)	0.48516752160641513
  (1, 136317)	0.5125311002730158
  (2, 71650)	0.5621198218335112
  (2, 99080)	0.2616858867134709
  (2, 13273)	0.3887964554215523
  (2, 8454)	0.35890991973427855
  :	:
  (319994, 65393)	0.4918118278116105
  (319995, 163

Training the Machine learning model

Logistic Regression

In [68]:
model = LogisticRegression(max_iter=1000)


In [70]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy Score

In [71]:
# accuracy scre on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [72]:
print("Accuracy Score of Training Data:",training_data_accuracy)

Accuracy Score of Training Data: 0.79871953125


In [78]:
# accuracy scre on testing data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [83]:
print("Accuracy Score of Testing Data:",test_data_accuracy)

Accuracy Score of Testing Data: 0.77668125


Model Accuracy = 77.7 %

saving the trained model

In [84]:
import pickle

In [85]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using the saved model for future prediction

In [87]:
#Loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [89]:
X_new = X_test[200]

In [90]:
print(Y_test[200])

1


In [92]:
prediction = loaded_model.predict(X_new)
print(prediction)

if prediction[0] == 0:
  print("Negative Tweet")
else:
  print("Positive Tweet")

[1]
Positive Tweet


In [93]:
X_new = X_test[3]
print(Y_test[3])


prediction = loaded_model.predict(X_new)
print(prediction)

if prediction[0] == 0:
  print("Negative Tweet")
else:
  print("Positive Tweet")

0
[0]
Negative Tweet
